In [7]:
!python -m pip install pandas numpy scikit-learn



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# ============================
# CELL 1: Import libraries
# ============================

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score


ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.
The following compiled module files exist, but seem incompatible
with with either python 'cpython-313' or the platform 'win32':

  * _multiarray_umath.cp313-win_amd64.lib
  * _multiarray_umath.cp313-win_amd64.pyd

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python 3.13 from "c:\Users\Shri\AppData\Local\Programs\Python\Python313\python3.13t.exe"
  * The NumPy version is: "2.4.1"

and make sure that they are the versions you expect.

Please carefully study the information and documentation linked above.
This is unlikely to be a NumPy issue but will be caused by a 

In [ ]:
import sys
print(sys.executable)


c:\Users\Shri\AppData\Local\Programs\Python\Python313\python3.13t.exe


In [ ]:
# ============================
# CELL 2: Load cleaned datasets
# ============================

enrol = pd.read_csv(
    "C:/Users/Pragya/Desktop/uidai-hackathon/aadhaar-enrollment/data/processed/clean_enrolment.csv"
)

demo = pd.read_csv(
    "C:/Users/Pragya/Desktop/uidai-hackathon/aadhaar-enrollment/data/processed/clean_demographic.csv"
)


NameError: name 'pd' is not defined

In [ ]:
# ============================
# CELL 3: Date conversion and sorting
# ============================

enrol['date'] = pd.to_datetime(enrol['date'])

enrol = enrol.sort_values(
    by=['state', 'district', 'date']
)


In [ ]:
# ============================
# CELL 4: Monthly enrolment aggregation
# ============================

monthly_enrol = (
    enrol.groupby(['state', 'district', 'date'])['enrolment']
    .sum()
    .reset_index()
)


In [ ]:
# ============================
# CELL 5: Month-over-month growth rate (TARGET)
# ============================

monthly_enrol['prev_enrolment'] = (
    monthly_enrol.groupby(['state', 'district'])['enrolment']
    .shift(1)
)

monthly_enrol['enrolment_growth_rate'] = (
    (monthly_enrol['enrolment'] - monthly_enrol['prev_enrolment']) /
    monthly_enrol['prev_enrolment']
)

monthly_enrol = monthly_enrol.dropna()


In [ ]:
# ============================
# CELL 6: Demographic feature preparation
# ============================

demo_features = (
    demo.groupby(['state', 'district'])[
        ['age_0_5', 'age_5_17', 'age_18_greater']
    ]
    .mean()
    .reset_index()
)


In [ ]:
# ============================
# CELL 7: Merge enrolment and demographics
# ============================

ml_data = pd.merge(
    monthly_enrol,
    demo_features,
    on=['state', 'district'],
    how='left'
)

ml_data = ml_data.dropna()


In [ ]:
# ============================
# CELL 8: Feature and target selection
# ============================

X = ml_data[
    ['prev_enrolment', 'age_0_5', 'age_5_17', 'age_18_greater']
]

y = ml_data['enrolment_growth_rate']


In [ ]:
# ============================
# CELL 9: Train-test split
# ============================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
# ============================
# CELL 10: Train Random Forest model
# ============================

model = RandomForestRegressor(
    n_estimators=100,
    random_state=42
)

model.fit(X_train, y_train)


In [ ]:
# ============================
# CELL 11: Model evaluation
# ============================

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("R2 Score:", r2)


In [ ]:
# ============================
# CELL 12: Predict growth rate for all districts
# ============================

ml_data['predicted_growth_rate'] = model.predict(X)


In [ ]:
# ============================
# CELL 13: Identify high-risk districts
# ============================

risk_districts = (
    ml_data.groupby(['state', 'district'])['predicted_growth_rate']
    .mean()
    .reset_index()
    .sort_values(by='predicted_growth_rate')
)

risk_districts.head(10)
